In [4]:
import numpy as np
from scipy.optimize import minimize, rosen
from sklearn.gaussian_process import GaussianProcessRegressor
from sklearn.model_selection import GridSearchCV
#from sklearn.gaussian_process.kernels import ConstantKernel, WhiteKernel, RBF, Matern, RationalQuadratic, ExpSineSquared, DotProduct, Sum, Product
from sklearn.gaussian_process.kernels import ConstantKernel as C, WhiteKernel as W, RBF, Matern, RationalQuadratic as RQ, ExpSineSquared as ESS, DotProduct as DP, Sum, Product
import warnings
#warnings.filterwarnings("ignore")

#===========================================================================================================#
#Obtaining initial values

np.random.seed(10003)
samples = 12
concentrations = 4
X = np.random.random(samples*concentrations).reshape(samples, concentrations)*10
y = rosen(X.T) * 1e-8

for i in range(len(y)):
    print(f'Sample {i+1}: {X[i].round(3)}, {y[i].round(6)}')


#===========================================================================================================#
#Parameters to Test

def make_bounds(lower, upper):
    bounds = []
    for low in lower:
        for up in upper:
                bounds.append((low, up))
    return bounds

all_kernels = []

kernel = [C(), W(), RBF(), Matern(), RQ(), ESS(), DP()]
operation = [Sum, Product]

upper_bound1 = [1e4, 1e5]
upper_bound2 = [1e2, 1e3, 1e4, 1e5]
upper_bound3 = [1e4, 1e5]
lower_bound1 = [1e-5, 1e-4, 1e-3, 1e-2, 1e-1, 1e0, 1e1, 1e2, 1e3]
lower_bound2 = [1e-5, 1e-4, 1e-3, 1e-2, 1e-1, 1e0, 1e1]
lower_bound3 = [1e-5, 1e-4, 1e-3, 1e-2, 1e-1, 1e0, 1e1, 1e2, 1e3]
            
kern1_bounds = make_bounds(lower_bound1, upper_bound1)
kern2_bounds = make_bounds(lower_bound2, upper_bound2)
kern3_bounds = make_bounds(lower_bound3, upper_bound3)

print(kern1_bounds, kern2_bounds, kern3_bounds)

#===========================================================================================================#
#Cross fold testing

model = GaussianProcessRegressor(normalize_y=True)
param_grid = [
    {'kernel': [Product(C(31.6**2), Sum(RBF(length_scale=10), DP(sigma_0=1e+04)))],
     'kernel__k1__constant_value_bounds': kern1_bounds,
     'kernel__k2__k1__length_scale_bounds': kern2_bounds,
     'kernel__k2__k2__sigma_0_bounds': kern3_bounds}
]

Matern(length_scale=1e+04, nu=1.5) * DotProduct(sigma_0=1e+04)
param_search = GridSearchCV(model, param_grid=param_grid, scoring='neg_mean_squared_error', cv=11, n_jobs=-1)

param_search.fit(X, -y)

best_parameters = param_search.best_params_
best_score = param_search.best_score_

print(best_parameters)
print(best_score)

Sample 1: [9.206 5.806 9.795 7.452], 0.014637
Sample 2: [9.123 5.265 8.88  5.74 ], 0.01178
Sample 3: [0.661 2.222 3.34  7.025], 2.3e-05
Sample 4: [2.022 2.641 2.202 2.139], 3.2e-05
Sample 5: [0.501 2.135 3.23  1.475], 8.6e-05
Sample 6: [0.363 1.749 2.184 1.032], 1.7e-05
Sample 7: [5.306 5.682 0.108 5.365], 0.00157
Sample 8: [0.608 8.135 1.662 2.234], 0.004223
Sample 9: [5.049 3.675 4.728 4.297], 0.000879
Sample 10: [7.569 8.265 5.136 5.122], 0.006847
Sample 11: [0.16  6.248 5.637 4.814], 0.001882
Sample 12: [3.649 6.6   2.837 8.771], 0.001705
{'kernel': 31.6**2 * RBF(length_scale=10) + DotProduct(sigma_0=1e+04), 'kernel__k1__constant_value_bounds': (10.0, 10000.0), 'kernel__k2__k1__length_scale_bounds': (10.0, 100.0), 'kernel__k2__k2__sigma_0_bounds': (0.001, 10000.0)}
-5.152893145395277e-06


{'kernel': 0.00316**2 + WhiteKernel(noise_level=1e-05), 'kernel__k1__constant_value': 1e-05, 'kernel__k2__noise_level': 1e-05}
-2.0958942474316883e-05

{'kernel': 316**2 * RBF(length_scale=10), 'kernel__k1__constant_value': 100000.0, 'kernel__k2__length_scale': 10.0}
-7.505383085897431e-06

{'kernel': 316**2 * Matern(length_scale=10, nu=2.5), 'kernel__k1__constant_value': 100000.0, 'kernel__k2__length_scale': 10.0, 'kernel__k2__nu': 2.5} 
-7.465419377027861e-06

{'kernel': 316**2 * RationalQuadratic(alpha=100, length_scale=100), 'kernel__k1__constant_value': 100000.0, 'kernel__k2__alpha': 100.0, 'kernel__k2__length_scale': 100.0} 
-6.059908345606381e-06

{'kernel': 0.00316**2 * ExpSineSquared(length_scale=1e+05, periodicity=1), 'kernel__k1__constant_value': 1e-05, 'kernel__k2__length_scale': 100000.0}
-8.892777082220134e-06

{'kernel': 31.6**2 * DotProduct(sigma_0=0.1), 'kernel__k1__constant_value': 1000.0, 'kernel__k2__sigma_0': 0.1} 
-1.2654839692894728e-05

{'kernel': WhiteKernel(noise_level=1e-05) + RBF(length_scale=1), 'kernel__k1__noise_level': 1e-05, 'kernel__k2__length_scale': 1.0} 
-8.594583355392786e-06

{'kernel': WhiteKernel(noise_level=1e-05) + Matern(length_scale=1, nu=inf), 'kernel__k1__noise_level': 1e-05, 'kernel__k2__length_scale': 1.0, 'kernel__k2__nu': inf} 
-8.5945833553882e-06

{'kernel': WhiteKernel(noise_level=1e-05) + RationalQuadratic(alpha=1e+05, length_scale=1), 'kernel__k1__noise_level': 1e-05, 'kernel__k2__alpha': 100000.0, 'kernel__k2__length_scale': 1.0}
-8.594585973429431e-06

{'kernel': WhiteKernel(noise_level=0.001) + ExpSineSquared(length_scale=100, periodicity=1), 'kernel__k1__noise_level': 0.001, 'kernel__k2__length_scale': 100.0}
-1.1335414305901361e-05

{'kernel': WhiteKernel(noise_level=0.01) + DotProduct(sigma_0=0.1), 'kernel__k1__noise_level': 0.01, 'kernel__k2__sigma_0': 0.1}
-1.25163147724037e-05

{'kernel': RBF(length_scale=1) + Matern(length_scale=100, nu=2.5), 'kernel__k1__length_scale': 1.0, 'kernel__k2__length_scale': 100.0, 'kernel__k2__nu': 2.5}
-7.593041249668872e-06

{'kernel': RBF(length_scale=1e+03) + RationalQuadratic(alpha=0.1, length_scale=1e-05), 'kernel__k1__length_scale': 1000.0, 'kernel__k2__alpha': 0.1, 'kernel__k2__length_scale': 1e-05}
-7.580777361184529e-06

{'kernel': RBF(length_scale=1) + ExpSineSquared(length_scale=1e+03, periodicity=1), 'kernel__k1__length_scale': 1.0, 'kernel__k2__length_scale': 1000.0}
-7.849433194598566e-06

{'kernel': RBF(length_scale=1) + DotProduct(sigma_0=1e-05), 'kernel__k1__length_scale': 1.0, 'kernel__k2__sigma_0': 1e-05}
-6.275795939795755e-06

{'kernel': Matern(length_scale=1e+04, nu=2.5) + RationalQuadratic(alpha=1e+03, length_scale=1e+03), 'kernel__k1__length_scale': 10000.0, 'kernel__k1__nu': 2.5, 'kernel__k2__alpha': 1000.0, 'kernel__k2__length_scale': 1000.0}
-7.6152566074880184e-06

{'kernel': Matern(length_scale=1, nu=inf) + ExpSineSquared(length_scale=1e+03, periodicity=1), 'kernel__k1__length_scale': 1.0, 'kernel__k1__nu': inf, 'kernel__k2__length_scale': 1000.0}
-7.849640310501145e-06

{'kernel': Matern(length_scale=1e+04, nu=1.5) * DotProduct(sigma_0=1e+04), 'kernel__k1__length_scale': 10000.0, 'kernel__k1__nu': 1.5, 'kernel__k2__sigma_0': 10000.0}
-5.981584114117213e-06

{'kernel': RationalQuadratic(alpha=1, length_scale=1) + ExpSineSquared(length_scale=100, periodicity=1), 'kernel__k1__alpha': 1.0, 'kernel__k1__length_scale': 1.0, 'kernel__k2__length_scale': 100.0}
-7.357665069268324e-06

{'kernel': RationalQuadratic(alpha=1e-05, length_scale=1e+04) * DotProduct(sigma_0=0.01), 'kernel__k1__alpha': 1e-05, 'kernel__k1__length_scale': 10000.0, 'kernel__k2__sigma_0': 0.01}
-6.036671579504781e-06

{'kernel': ExpSineSquared(length_scale=0.1, periodicity=1) * DotProduct(sigma_0=1), 'kernel__k1__length_scale': 0.1, 'kernel__k2__sigma_0': 1.0}
-1.3753347464818623e-05

{
    'kernel': [Sum(C(), W()), Product(C(), W())],
    'kernel__k1__constant_value': [1e-5, 1e-4, 1e-3, 1e-2, 1e-1, 1e0, 1e1, 1e2, 1e3, 1e4, 1e5],
    'kernel__k2__noise_level': [1e-5, 1e-4, 1e-3, 1e-2, 1e-1, 1e0, 1e1, 1e2, 1e3, 1e4, 1e5]
}

{
    'kernel': [Sum(C(), RBF()), Product(C(), RBF())],
    'kernel__k1__constant_value': [1e-5, 1e-4, 1e-3, 1e-2, 1e-1, 1e0, 1e1, 1e2, 1e3, 1e4, 1e5],
    'kernel__k2__length_scale': [1e-5, 1e-4, 1e-3, 1e-2, 1e-1, 1e0, 1e1, 1e2, 1e3, 1e4, 1e5]
}

{
    'kernel': [Sum(C(), Matern()), Product(C(), Matern())],
    'kernel__k1__constant_value': [1e-5, 1e-4, 1e-3, 1e-2, 1e-1, 1e0, 1e1, 1e2, 1e3, 1e4, 1e5],
    'kernel__k2__length_scale': [1e-5, 1e-4, 1e-3, 1e-2, 1e-1, 1e0, 1e1, 1e2, 1e3, 1e4, 1e5],
    'kernel__k2__nu': [0.5, 1.5, 2.5, float('inf')]
    
}

{
    'kernel': [Sum(C(), RQ()), Product(C(), RQ())],
    'kernel__k1__constant_value': [1e-5, 1e-4, 1e-3, 1e-2, 1e-1, 1e0, 1e1, 1e2, 1e3, 1e4, 1e5],
    'kernel__k2__length_scale': [1e-5, 1e-4, 1e-3, 1e-2, 1e-1, 1e0, 1e1, 1e2, 1e3, 1e4, 1e5],
    'kernel__k2__alpha': [1e-5, 1e-4, 1e-3, 1e-2, 1e-1, 1e0, 1e1, 1e2, 1e3, 1e4, 1e5]

}

{
    'kernel': [Sum(C(), ESS()), Product(C(), ESS())],
    'kernel__k1__constant_value': [1e-5, 1e-4, 1e-3, 1e-2, 1e-1, 1e0, 1e1, 1e2, 1e3, 1e4, 1e5],
    'kernel__k2__length_scale': [1e-5, 1e-4, 1e-3, 1e-2, 1e-1, 1e0, 1e1, 1e2, 1e3, 1e4, 1e5]
    
}

{
    'kernel': [Sum(C(), DP()), Product(C(), DP())],
    'kernel__k1__constant_value': [1e-5, 1e-4, 1e-3, 1e-2, 1e-1, 1e0, 1e1, 1e2, 1e3, 1e4, 1e5],
    'kernel__k2__sigma_0': [1e-5, 1e-4, 1e-3, 1e-2, 1e-1, 1e0, 1e1, 1e2, 1e3, 1e4, 1e5]
    
}

{
    'kernel': [Sum(W(), RBF()), Product(W(), RBF())],
    'kernel__k1__noise_level': [1e-5, 1e-4, 1e-3, 1e-2, 1e-1, 1e0, 1e1, 1e2, 1e3, 1e4, 1e5],
    'kernel__k2__length_scale': [1e-5, 1e-4, 1e-3, 1e-2, 1e-1, 1e0, 1e1, 1e2, 1e3, 1e4, 1e5]
    
}

{
    'kernel': [Sum(W(), Matern()), Product(W(), Matern())],
    'kernel__k1__noise_level': [1e-5, 1e-4, 1e-3, 1e-2, 1e-1, 1e0, 1e1, 1e2, 1e3, 1e4, 1e5],
    'kernel__k2__length_scale': [1e-5, 1e-4, 1e-3, 1e-2, 1e-1, 1e0, 1e1, 1e2, 1e3, 1e4, 1e5],
    'kernel__k2__nu': [0.5, 1.5, 2.5, float('inf')]
    
}

{
    'kernel': [Sum(W(), RQ()), Product(W(), RQ())],
    'kernel__k1__noise_level': [1e-5, 1e-4, 1e-3, 1e-2, 1e-1, 1e0, 1e1, 1e2, 1e3, 1e4, 1e5],
    'kernel__k2__length_scale': [1e-5, 1e-4, 1e-3, 1e-2, 1e-1, 1e0, 1e1, 1e2, 1e3, 1e4, 1e5],
    'kernel__k2__alpha': [1e-5, 1e-4, 1e-3, 1e-2, 1e-1, 1e0, 1e1, 1e2, 1e3, 1e4, 1e5]
    
}

{
    'kernel': [Sum(W(), ESS()), Product(W(), ESS())],
    'kernel__k1__noise_level': [1e-5, 1e-4, 1e-3, 1e-2, 1e-1, 1e0, 1e1, 1e2, 1e3, 1e4, 1e5],
    'kernel__k2__length_scale': [1e-5, 1e-4, 1e-3, 1e-2, 1e-1, 1e0, 1e1, 1e2, 1e3, 1e4, 1e5]
    
}

{
    'kernel': [Sum(W(), DP()), Product(W(), DP())],
    'kernel__k1__noise_level': [1e-5, 1e-4, 1e-3, 1e-2, 1e-1, 1e0, 1e1, 1e2, 1e3, 1e4, 1e5],
    'kernel__k2__sigma_0': [1e-5, 1e-4, 1e-3, 1e-2, 1e-1, 1e0, 1e1, 1e2, 1e3, 1e4, 1e5]
    
}

{
    'kernel': [Sum(RBF(), Matern()), Product(RBF(), Matern())],
    'kernel__k1__length_scale': [1e-5, 1e-4, 1e-3, 1e-2, 1e-1, 1e0, 1e1, 1e2, 1e3, 1e4, 1e5],
    'kernel__k2__length_scale': [1e-5, 1e-4, 1e-3, 1e-2, 1e-1, 1e0, 1e1, 1e2, 1e3, 1e4, 1e5],
    'kernel__k2__nu': [0.5, 1.5, 2.5, float('inf')]
    
}

{
    'kernel': [Sum(RBF(), RQ()), Product(RBF(), RQ())],
    'kernel__k1__length_scale': [1e-5, 1e-4, 1e-3, 1e-2, 1e-1, 1e0, 1e1, 1e2, 1e3, 1e4, 1e5],
    'kernel__k2__length_scale': [1e-5, 1e-4, 1e-3, 1e-2, 1e-1, 1e0, 1e1, 1e2, 1e3, 1e4, 1e5],
    'kernel__k2__alpha': [1e-5, 1e-4, 1e-3, 1e-2, 1e-1, 1e0, 1e1, 1e2, 1e3, 1e4, 1e5]
    
}

{
    'kernel': [Sum(RBF(), ESS()), Product(RBF(), ESS())],
    'kernel__k1__length_scale': [1e-5, 1e-4, 1e-3, 1e-2, 1e-1, 1e0, 1e1, 1e2, 1e3, 1e4, 1e5],
    'kernel__k2__length_scale': [1e-5, 1e-4, 1e-3, 1e-2, 1e-1, 1e0, 1e1, 1e2, 1e3, 1e4, 1e5]
}

{
    'kernel': [Sum(RBF(), DP()), Product(RBF(), DP())],
    'kernel__k1__length_scale': [1e-5, 1e-4, 1e-3, 1e-2, 1e-1, 1e0, 1e1, 1e2, 1e3, 1e4, 1e5],
    'kernel__k2__sigma_0': [1e-5, 1e-4, 1e-3, 1e-2, 1e-1, 1e0, 1e1, 1e2, 1e3, 1e4, 1e5]
    
}

{
    'kernel': [Sum(Matern(), RQ()), Product(Matern(), RQ())],
    'kernel__k1__length_scale': [1e-5, 1e-4, 1e-3, 1e-2, 1e-1, 1e0, 1e1, 1e2, 1e3, 1e4, 1e5],
    'kernel__k1__nu': [0.5, 1.5, 2.5, float('inf')],
    'kernel__k2__length_scale': [1e-5, 1e-4, 1e-3, 1e-2, 1e-1, 1e0, 1e1, 1e2, 1e3, 1e4, 1e5],
    'kernel__k2__alpha': [1e-5, 1e-4, 1e-3, 1e-2, 1e-1, 1e0, 1e1, 1e2, 1e3, 1e4, 1e5]
    
}

{
    'kernel': [Sum(Matern(), ESS()), Product(Matern(), ESS())],
    'kernel__k1__length_scale': [1e-5, 1e-4, 1e-3, 1e-2, 1e-1, 1e0, 1e1, 1e2, 1e3, 1e4, 1e5],
    'kernel__k1__nu': [0.5, 1.5, 2.5, float('inf')],
    'kernel__k2__length_scale': [1e-5, 1e-4, 1e-3, 1e-2, 1e-1, 1e0, 1e1, 1e2, 1e3, 1e4, 1e5]
    
}

{
    'kernel': [Sum(Matern(), DP()), Product(Matern(), DP())],
    'kernel__k1__length_scale': [1e-5, 1e-4, 1e-3, 1e-2, 1e-1, 1e0, 1e1, 1e2, 1e3, 1e4, 1e5],
    'kernel__k1__nu': [0.5, 1.5, 2.5, float('inf')],
    'kernel__k2__sigma_0': [1e-5, 1e-4, 1e-3, 1e-2, 1e-1, 1e0, 1e1, 1e2, 1e3, 1e4, 1e5]
    
}

{
    'kernel': [Sum(RQ(), ESS()), Product(RQ(), ESS())],
    'kernel__k1__length_scale': [1e-5, 1e-4, 1e-3, 1e-2, 1e-1, 1e0, 1e1, 1e2, 1e3, 1e4, 1e5],
    'kernel__k1__alpha': [1e-5, 1e-4, 1e-3, 1e-2, 1e-1, 1e0, 1e1, 1e2, 1e3, 1e4, 1e5],
    'kernel__k2__length_scale': [1e-5, 1e-4, 1e-3, 1e-2, 1e-1, 1e0, 1e1, 1e2, 1e3, 1e4, 1e5]
    
}

{
    'kernel': [Sum(RQ(), DP()), Product(RQ(), DP())],
    'kernel__k1__length_scale': [1e-5, 1e-4, 1e-3, 1e-2, 1e-1, 1e0, 1e1, 1e2, 1e3, 1e4, 1e5],
    'kernel__k1__alpha': [1e-5, 1e-4, 1e-3, 1e-2, 1e-1, 1e0, 1e1, 1e2, 1e3, 1e4, 1e5],
    'kernel__k2__sigma_0': [1e-5, 1e-4, 1e-3, 1e-2, 1e-1, 1e0, 1e1, 1e2, 1e3, 1e4, 1e5]
    
}

{
    'kernel': [Sum(ESS(), DP()), Product(ESS(), DP())],
    'kernel__k1__length_scale': [1e-5, 1e-4, 1e-3, 1e-2, 1e-1, 1e0, 1e1, 1e2, 1e3, 1e4, 1e5],
    'kernel__k2__sigma_0': [1e-5, 1e-4, 1e-3, 1e-2, 1e-1, 1e0, 1e1, 1e2, 1e3, 1e4, 1e5]
    
}

{'kernel':  [Sum(ConstantKernel(0.00316**2), WhiteKernel(noise_level=1e-05))], 'kernel__k1__constant_value': [1e-05], 'kernel__k2__noise_level': [1e-05]},


    {'kernel':  [Product(ConstantKernel(316**2), RBF(length_scale=10))], 'kernel__k1__constant_value': [100000.0], 'kernel__k2__length_scale': [10.0]},


    {'kernel':  [Product(ConstantKernel(316**2), Matern(length_scale=10, nu=2.5))], 'kernel__k1__constant_value': [100000.0], 'kernel__k2__length_scale': [10.0], 'kernel__k2__nu': [2.5]},


    {'kernel':  [Product(ConstantKernel(316**2), RationalQuadratic(alpha=100, length_scale=100))], 'kernel__k1__constant_value': [100000.0], 'kernel__k2__alpha': [100.0], 'kernel__k2__length_scale': [100.0]},


    {'kernel':  [Product(ConstantKernel(0.00316**2), ExpSineSquared(length_scale=1e+05, periodicity=1))], 'kernel__k1__constant_value': [1e-05], 'kernel__k2__length_scale': [100000.0]},


    {'kernel':  [Product(ConstantKernel(31.6**2), DotProduct(sigma_0=0.1))], 'kernel__k1__constant_value': [1000.0], 'kernel__k2__sigma_0': [0.1]},


    {'kernel':  [Sum(WhiteKernel(noise_level=1e-05), RBF(length_scale=1))], 'kernel__k1__noise_level': [1e-05], 'kernel__k2__length_scale': [1.0]},


    {'kernel':  [Sum(WhiteKernel(noise_level=1e-05), Matern(length_scale=1, nu=float('inf')))], 'kernel__k1__noise_level': [1e-05], 'kernel__k2__length_scale': [1.0], 'kernel__k2__nu': [float('inf')]},


    {'kernel':  [Sum(WhiteKernel(noise_level=1e-05), RationalQuadratic(alpha=1e+05, length_scale=1))], 'kernel__k1__noise_level': [1e-05], 'kernel__k2__alpha': [100000.0], 'kernel__k2__length_scale': [1.0]},


    {'kernel':  [Sum(WhiteKernel(noise_level=0.001), ExpSineSquared(length_scale=100, periodicity=1))], 'kernel__k1__noise_level': [0.001], 'kernel__k2__length_scale': [100.0]},


    {'kernel':  [Sum(WhiteKernel(noise_level=0.01), DotProduct(sigma_0=0.1))], 'kernel__k1__noise_level': [0.01], 'kernel__k2__sigma_0': [0.1]},


    {'kernel':  [Sum(RBF(length_scale=1), Matern(length_scale=100, nu=2.5))], 'kernel__k1__length_scale': [1.0], 'kernel__k2__length_scale': [100.0], 'kernel__k2__nu': [2.5]},


    {'kernel':  [Sum(RBF(length_scale=1e+03), RationalQuadratic(alpha=0.1, length_scale=1e-05))], 'kernel__k1__length_scale': [1000.0], 'kernel__k2__alpha': [0.1], 'kernel__k2__length_scale': [1e-05]},


    {'kernel':  [Sum(RBF(length_scale=1), ExpSineSquared(length_scale=1e+03, periodicity=1))], 'kernel__k1__length_scale': [1.0], 'kernel__k2__length_scale': [1000.0]},


    {'kernel':  [Sum(RBF(length_scale=1), DotProduct(sigma_0=1e-05))], 'kernel__k1__length_scale': [1.0], 'kernel__k2__sigma_0': [1e-05]},


    {'kernel':  [Sum(Matern(length_scale=1e+04, nu=2.5), RationalQuadratic(alpha=1e+03, length_scale=1e+03))], 'kernel__k1__length_scale': [10000.0], 'kernel__k1__nu': [2.5], 'kernel__k2__alpha': [1000.0], 'kernel__k2__length_scale': [1000.0]},


    {'kernel':  [Sum(Matern(length_scale=1, nu=float('inf')), ExpSineSquared(length_scale=1e+03, periodicity=1))], 'kernel__k1__length_scale': [1.0], 'kernel__k1__nu': [float('inf')], 'kernel__k2__length_scale': [1000.0]},


    {'kernel':  [Product(Matern(length_scale=1e+04, nu=1.5), DotProduct(sigma_0=1e+04))], 'kernel__k1__length_scale': [10000.0], 'kernel__k1__nu': [1.5], 'kernel__k2__sigma_0': [10000.0]},


    {'kernel':  [Sum(RationalQuadratic(alpha=1, length_scale=1), ExpSineSquared(length_scale=100, periodicity=1))], 'kernel__k1__alpha': [1.0], 'kernel__k1__length_scale': [1.0], 'kernel__k2__length_scale': [100.0]},


    {'kernel':  [Product(RationalQuadratic(alpha=1e-05, length_scale=1e+04), DotProduct(sigma_0=0.01))], 'kernel__k1__alpha': [1e-05], 'kernel__k1__length_scale': [10000.0], 'kernel__k2__sigma_0': [0.01]},


    {'kernel':  [Product(ExpSineSquared(length_scale=0.1, periodicity=1), DotProduct(sigma_0=1))], 'kernel__k1__length_scale': [0.1], 'kernel__k2__sigma_0': [1.0]}

    {'kernel': Matern(length_scale=1e+04, nu=1.5) * DotProduct(sigma_0=1e+04), 'kernel__k1__length_scale': 10000.0, 'kernel__k1__nu': 1.5, 'kernel__k2__sigma_0': 10000.0}
-5.981584114117213e-06


{'kernel': 1**2 * RBF(length_scale=1) + DotProduct(sigma_0=1)}
-5.534414196866314e-06

{'kernel': 31.6**2 * RBF(length_scale=10) + DotProduct(sigma_0=1e+04), 'kernel__k1__constant_value_bounds': (10.0, 10000.0), 'kernel__k2__k1__length_scale_bounds': (10.0, 100.0), 'kernel__k2__k2__sigma_0_bounds': (0.001, 10000.0)}
        'kernel': [Product(C(31.6**2), Sum(RBF(length_scale=10), DP(sigma_0=1e+04)))],
        'kernel__k1__constant_value_bounds': kern1_bounds,
        'kernel__k2__k1__length_scale_bounds': kern2_bounds,
        'kernel__k2__k2__sigma_0_bounds': kern3_bounds
kernel = 31.6**2 * RBF(length_scale=10, constant_value_bounds=(10.0, 10000.0)) + DotProduct(sigma_0=1e+04, sigma_0_bounds=(0.001, 10000.0))
-5.152893145395277e-06